# 크로마
- 크로마는 간단하고 직관적인 인터페이스를 제공하므로, 사용자들이 쉽게 설정하고 사용할 수 있어 빠른 프로토타이핑에 적합합니다.
- 다양한 데이터 타입을 지원해 데이터 확장성이 뛰어납니다.
- 주로 단일 노드 환경에서 사용하기때문에 대규모 데이터셋을 다룰 때는 성능에 한계가 있을 수 있습니다.

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma

In [2]:
# PDF 파일 로드
loader = PyPDFLoader("2024_KB_부동산_보고서_최종.pdf")
pages = loader.load()
print(f"청크의 수: {len(pages)}")

청크의 수: 84


- chunk_size=1000 : 각 청크는 1000자로 분할
- chunk_overlap=200 : 청크 간 200자씩 겹침

In [3]:
# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
splits = text_splitter.split_documents(pages)
print(f"분할된 청크의 수: {len(splits)}")

분할된 청크의 수: 135


In [4]:
# 각 청크의 길이(문자 수)를 저장한 리스트 생성
chunk_lengths = [len(split.page_content) for split in splits]
max_length = max(chunk_lengths)
min_length = min(chunk_lengths)
avg_length = sum(chunk_lengths) / len(chunk_lengths)

print(f"최대 청크 길이: {max_length}")
print(f"최소 청크 길이: {min_length}")
print(f"평균 청크 길이: {avg_length}")

최대 청크 길이: 1000
최소 청크 길이: 56
평균 청크 길이: 674.9481481481481


### 크로마 데이터베이스 저장
persist_directory의 기본값은 'None'이며, 이 경우 크로마는 데이터를 메모리에만 저장하므로 프로그램이 실행되는 동안에만 데이터가 유지됩니다.  
persist_directory에 경로를 설정하면 크로마가 해당 디렉토리에서 데이터를 읽거나 쓸 수 있으며, 이를 통해 데이터베이스를 디스크에 저장하여 코드 실행이 종료되어도 데이터를 유지할 수 있습니다.  
이 설정이 없으면 메모리에만 남아 세션 종료시 삭제되므로, 데이터를 장기적으로 보존하고 지속적으로 사용하려면 persist_directory 경로를 설정해야 합니다.


In [8]:
# 임베딩 모델 초기화
embedding_function = OllamaEmbeddings(model="bge-m3")

In [ ]:
# Chroma DB 생성 및 데이터 저장
persist_directory = "chroma_db"
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding_function,
    persist_directory=persist_directory,
)
print(f"문서의 수:", vectordb._collection.count())

문서의 수: 135


- Chroma.from_documents() 메서드를 반복적으로 호출할 경우, 메모리가 중복으로 쌓일 수 있으므로 주의해야 합니다.

In [9]:
# Chroma DB 불러오기
vectordb = Chroma(
    embedding_function=embedding_function,
    persist_directory=persist_directory,
)
print(f"문서의 수:", vectordb._collection.count())

문서의 수: 135


- 데이터베이스 객체를 생성한 후, 사용자 입력과 유사한 문서를 찾으려면 similarity_search()를 사용합니다.

In [13]:
# similarity 검색
question = "수도권 주택 매매 전망"
top_three_docs = vectordb.similarity_search(question, k=2)
for i, doc in enumerate(top_three_docs, 1):
    print(f"문서 {i}:")
    print(f"내용: {doc.page_content[:150]}...") # 텍스트의 처음 150자만 출력
    print(f"메타데이터: {doc.metadata}")
    print("-" * 50)

문서 1:
내용: 40 
2024 KB 부동산 보고서: 수도권 주택시장 점검 
 
 
 
주택 매매가격은 2023년 상반기 이후 회복세를 보였으나 상승폭이 크지 않았으며 12월 다시 하락세
로 전환되었다. 2023년 연간 주택 매매가격은 4.64% 하락하면서 2022년(1.83% 하락)...
메타데이터: {'author': '손은경', 'creationdate': '2024-03-04T15:30:01+09:00', 'creator': 'Microsoft® Word 2016', 'moddate': '2024-03-04T15:30:01+09:00', 'page': 46, 'page_label': '47', 'producer': 'Microsoft® Word 2016', 'source': '2024_KB_부동산_보고서_최종.pdf', 'title': 'Morning Meeting', 'total_pages': 84}
--------------------------------------------------
문서 2:
내용: 28 
2024 KB 부동산 보고서: 주택시장 설문조사 
 
 
1) 2024년 주택시장 전망 
■ 주택 매매가격, 2024년에도 하락 전망 우세한 가운데 상승 전망 2023년 대비 증가 
부동산시장 전문가와 공인중개사, 자산관리전문가(PB)를 대상으로 한 설문조사 결...
메타데이터: {'author': '손은경', 'creationdate': '2024-03-04T15:30:01+09:00', 'creator': 'Microsoft® Word 2016', 'moddate': '2024-03-04T15:30:01+09:00', 'page': 34, 'page_label': '35', 'producer': 'Microsoft® Word 2016', 'source': '2024_KB_부동산_보고서_최종.pdf', 'title': 'Morning Meeting', 'total_pages': 84}
-----------------------------

In [14]:
# similarity 검색(with relevance scores)
question = "수도권 주택 매매 전망"
top_three_docs = vectordb.similarity_search_with_relevance_scores(question, k=3)
for i, doc in enumerate(top_three_docs, 1):
    print(f"문서 {i}:")
    print(f"유사도 점수: {doc[1]}")
    print(f"내용: {doc[0].page_content[:150]}...") # 텍스트의 처음 150자만 출력
    print(f"메타데이터: {doc[0].metadata}")
    print("-" * 50)

문서 1:
유사도 점수: 0.5542737276432206
내용: 40 
2024 KB 부동산 보고서: 수도권 주택시장 점검 
 
 
 
주택 매매가격은 2023년 상반기 이후 회복세를 보였으나 상승폭이 크지 않았으며 12월 다시 하락세
로 전환되었다. 2023년 연간 주택 매매가격은 4.64% 하락하면서 2022년(1.83% 하락)...
메타데이터: {'author': '손은경', 'creationdate': '2024-03-04T15:30:01+09:00', 'creator': 'Microsoft® Word 2016', 'moddate': '2024-03-04T15:30:01+09:00', 'page': 46, 'page_label': '47', 'producer': 'Microsoft® Word 2016', 'source': '2024_KB_부동산_보고서_최종.pdf', 'title': 'Morning Meeting', 'total_pages': 84}
--------------------------------------------------
문서 2:
유사도 점수: 0.5453653590077154
내용: 28 
2024 KB 부동산 보고서: 주택시장 설문조사 
 
 
1) 2024년 주택시장 전망 
■ 주택 매매가격, 2024년에도 하락 전망 우세한 가운데 상승 전망 2023년 대비 증가 
부동산시장 전문가와 공인중개사, 자산관리전문가(PB)를 대상으로 한 설문조사 결...
메타데이터: {'author': '손은경', 'creationdate': '2024-03-04T15:30:01+09:00', 'creator': 'Microsoft® Word 2016', 'moddate': '2024-03-04T15:30:01+09:00', 'page': 34, 'page_label': '35', 'producer': 'Microsoft® Word 2016', 'source': '2024_KB_부동산_보고서_최종.pdf', 'title': 'Morning Meet

# 파이스
- 파이스는 페이스북 AI리서치 팀이 개발한 고성능 벡터 검색 라이브러리입니다.
- 대규모 데이터셋에서 유사성 검색과 클러스터링 작업을 효율적으로 수행할 수 있도록 설계되었습니다.
- 파이스는 GPU가속을 통해 수백만 개 이상의 고차원 벡터를 처리하여 매우 빠른 속도를 제공합니다.
- 고급 기능을 활용하려면 설정이 복잡하고, 분산 시스템에서는 제약이 있을 수 있다는 단점이 있습니다.
- GPU 사용이 가능한 환경이라면 faiss-gpu, 그렇지 않다면 faiss-cpu를 설치합니다.

In [15]:
from langchain_community.vectorstores import FAISS

In [ ]:
# 파이스 DB 생성
faiss_db = FAISS.from_documents(documents=splits, embedding=embedding_function)
print(f"문서의 수: {faiss_db.index.ntotal}")

문서의 수: 135


- faiss_db.save_local('파일 경로 및 파일명')을 사용하면 파이스 벡터 데이터베이스를 파일로 저장할 수 있습니다.
- FAISS.load_local('파일 경로 및 파일명')을 사용하면 저장한 벡터데이터베이스를 로드할 수 있습니다.
- allow_dangerous_deserialization=True 옵션은 파일을 읽을 때 보안 경고를 무시하고 객체를 복원하도록 하는 설정입니다.
- 파이썬에서는 파일을 객체로 변환하는 과정에서 보안 위험이 있을 경우 파일 읽기를 거부할 수 있습니다.

In [ ]:
# 파이스 DB 저장하기
faiss_directory = "faiss_index"
faiss_db.save_local(faiss_directory)

# 파이스 DB 불러오기
new_db_faiss = FAISS.load_local(faiss_directory,
                                embedding_function,
                                allow_dangerous_deserialization=True)

In [18]:
# 검색할 질문 정의
question = "수도권 주택 매매 전망"
# similarity 검색
docs = new_db_faiss.similarity_search(question)
for i, doc in enumerate(docs, 1):
    print(f"문서 {i}:")
    print(f"내용: {doc.page_content[:150]}...") # 텍스트의 처음 150자만 출력
    print(f"메타데이터: {doc.metadata}")
    print("-" * 50)

문서 1:
내용: 40 
2024 KB 부동산 보고서: 수도권 주택시장 점검 
 
 
 
주택 매매가격은 2023년 상반기 이후 회복세를 보였으나 상승폭이 크지 않았으며 12월 다시 하락세
로 전환되었다. 2023년 연간 주택 매매가격은 4.64% 하락하면서 2022년(1.83% 하락)...
메타데이터: {'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-03-04T15:30:01+09:00', 'title': 'Morning Meeting', 'author': '손은경', 'moddate': '2024-03-04T15:30:01+09:00', 'source': '2024_KB_부동산_보고서_최종.pdf', 'total_pages': 84, 'page': 46, 'page_label': '47'}
--------------------------------------------------
문서 2:
내용: 28 
2024 KB 부동산 보고서: 주택시장 설문조사 
 
 
1) 2024년 주택시장 전망 
■ 주택 매매가격, 2024년에도 하락 전망 우세한 가운데 상승 전망 2023년 대비 증가 
부동산시장 전문가와 공인중개사, 자산관리전문가(PB)를 대상으로 한 설문조사 결...
메타데이터: {'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2024-03-04T15:30:01+09:00', 'title': 'Morning Meeting', 'author': '손은경', 'moddate': '2024-03-04T15:30:01+09:00', 'source': '2024_KB_부동산_보고서_최종.pdf', 'total_pages': 84, 'page': 34, 'page_label': '35'}
-----------------------------

- 챗봇에서 벡터 DB를 사용할 때는 as_retriever() 메서드를 호출하여 검색기로 활용할 수 있습니다.